<a href="https://colab.research.google.com/github/kakans/LLM/blob/main/Fast_Inference_PEFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqqq pip --progress-bar off
#!pip install -qqq bitsandbytes==0.39.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.30.0 --progress-bar off
!pip install -qqq accelerate==0.20.3 --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/peft.git --progress-bar off
!pip install -qqq datasets==2.12.0 --progress-bar off
!pip install -qqq loralib==0.1.1 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!python -m pip install bitsandbytes --prefer-binary --extra-index-url=https://jllllll.github.io/bitsandbytes-windows-webui



  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://jllllll.github.io/bitsandbytes-windows-webui
   ━━━

In [ ]:
import torch
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b", padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [ ]:
DEVICE = "cuda:0"

prompt = f"""
: What is your return policy?
:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
!python -m bitsandbytes

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++ BUG REPORT INFORMATION ++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++ /usr/local CUDA PATHS +++++++++++++++++++
/usr/local/lib/python3.10/dist-packages/torch/lib/libtorch_cuda_linalg.so
/usr/local/lib/python3.10/dist-packages/torch/lib/libtorch_cuda.so
/usr/local/lib/python3.10/dist-packages/torch/lib/libc10_cuda.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda117_nocublaslt.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda114_nocublaslt.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda122_nocublaslt.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda121.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda117.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda110_nocublaslt.so
/usr/local/lib/python3.10

In [ ]:
MODEL_ID = "curiousily/falcon-7b-qlora-chat-support-bot-faq"
config = PeftConfig.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model = PeftModel.from_pretrained(model, MODEL_ID)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 20
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%timeit -r 5
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
        do_sample=False,
        use_cache=True,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

: What is your return policy?
: We offer a 30-day return policy on all products. Please contact our customer support team with
: What is your return policy?
: We offer a 30-day return policy on all products. Please contact our customer support team with
: What is your return policy?
: We offer a 30-day return policy on all products. Please contact our customer support team with
: What is your return policy?
: We offer a 30-day return policy on all products. Please contact our customer support team with
: What is your return policy?
: We offer a 30-day return policy on all products. Please contact our customer support team with
: What is your return policy?
: We offer a 30-day return policy on all products. Please contact our customer support team with
2.08 s ± 15.4 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


4 BIT QLOAR

In [ ]:
MODEL_ID = "curiousily/falcon-7b-qlora-chat-support-bot-faq"
config = PeftConfig.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    device_map="auto",
    trust_remote_code=True,
    load_in_4bit=True,
)

model = PeftModel.from_pretrained(model, MODEL_ID)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 20
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%timeit -r 5

with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
        do_sample=False,
        use_cache=True,
    )

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


2.16 s ± 13.8 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [ ]:
%%timeit -r 5
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
        do_sample=False,
        use_cache=True,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

: What is your return policy?
: If you need to return an item, please contact our customer support team first. We will guide you
: What is your return policy?
: If you need to return an item, please contact our customer support team first. We will guide you
: What is your return policy?
: If you need to return an item, please contact our customer support team first. We will guide you
: What is your return policy?
: If you need to return an item, please contact our customer support team first. We will guide you
: What is your return policy?
: If you need to return an item, please contact our customer support team first. We will guide you
: What is your return policy?
: If you need to return an item, please contact our customer support team first. We will guide you
2.21 s ± 35.3 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
